In [25]:
import pandas as pd
import numpy as np 
import matplotlib as mpl 
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
%matplotlib inline
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error, accuracy_score

In [26]:
nba_shots_Chicago_="shot log CHI.csv"

In [27]:
shots = pd.read_csv(nba_shots_Chicago_)
shots = shots[(shots['self previous shot'].notnull()) &
                                (shots['opponent previous shot'].notnull()) & 
                                (shots['location x'].notnull()) & 
                                (shots['location y'].notnull()) & 
                                (shots['quarter'].notnull()) & 
                                (shots['self previous shot'].notnull()) &
                                (shots['opponent previous shot'].notnull()) & 
                                (shots['time'].notnull()) &
                                (shots['time from last shot']) & 
                                (shots['player position'].notnull())]


In [28]:
shots.columns.values
# 'location x', 'location y', 'quarter', 'self previous shot', 'opponent previous shot', 'time', 'time from last shot', 'player position', 'home game', 'shot type', 'points', 'time', 'time from last shot', 'quarter', 'current shot outcome'


array(['self previous shot', 'player position', 'home game', 'location x',
       'opponent previous shot', 'home team', 'shot type', 'points',
       'away team', 'location y', 'time', 'date', 'shoot player',
       'time from last shot', 'quarter', 'current shot outcome'],
      dtype=object)

In [29]:
shots[['time']].describe()

,time
count,6744
unique,695
top,12:00
freq,99


In [30]:
shots[['location x', 'location y', 'quarter', 'player position', 'home game', 'self previous shot', 'shoot player', 'opponent previous shot']]

,location x,location y,quarter,player position,home game,self previous shot,shoot player,opponent previous shot
1,64.0,215.0,1,PF,Yes,SCORED,Taj Gibson,MISSED
2,67.0,259.0,1,PG,Yes,MISSED,Rajon Rondo,MISSED
3,102.0,243.0,1,C,Yes,MISSED,Robin Lopez,MISSED
4,241.0,210.0,1,SG,Yes,MISSED,Dwyane Wade,SCORED
5,239.0,351.0,1,SG,Yes,MISSED,Jimmy Butler,SCORED
...,...,...,...,...,...,...,...,...
7134,640.0,212.0,4,G,Yes,SCORED,Denzel Valentine,MISSED
7135,675.0,113.0,4,G,Yes,SCORED,Denzel Valentine,MISSED
7136,848.0,112.0,4,SG,Yes,MISSED,Anthony Morrow,SCORED
7137,691.0,99.0,4,PF,Yes,SCORED,Bobby Portis,MISSED


In [31]:
from sklearn import preprocessing


# Model for predicting which position shot the ball:
# explanatory variables
# 'location x', 'location y', 'quarter', 'self previous shot', 'opponent previous shot', 'time', 'time from last shot'
# prediction variables:
# 'player position'

position_model = shots[['location x', 'location y', 'quarter', 'self previous shot', 'opponent previous shot', 'time', 'time from last shot','player position']]

# Encode our data
shot_encoder = preprocessing.LabelEncoder()
shot_encoder.fit(position_model['self previous shot'].unique().tolist())
print(list(shot_encoder.classes_))
position_model['enc_self previous shot'] = shot_encoder.transform(position_model['self previous shot'])


opp_shot_encoder = preprocessing.LabelEncoder()
opp_shot_encoder.fit(position_model['opponent previous shot'].unique().tolist())
print(list(opp_shot_encoder.classes_))
position_model['enc_opponent previous shot'] = opp_shot_encoder.transform(position_model['opponent previous shot'])


position_encoder = preprocessing.LabelEncoder()
position_encoder.fit(position_model['player position'].unique().tolist())
print(list(position_encoder.classes_))
position_model['enc_player position'] = position_encoder.transform(position_model['player position'])


position_model['enc_time'] = position_model['time'].str.replace(':', '.').astype(float)
position_model['location x'] = position_model['location x'].astype(float)
position_model['location y'] = position_model['location y'].astype(float)
position_model['quarter'] = position_model['quarter'].astype(float)

scaled_model = position_model[['location x', 'location y', 'enc_self previous shot', 'enc_opponent previous shot', 'enc_time', 'quarter']]

scaler = preprocessing.StandardScaler().fit(scaled_model)
scaled_model = scaler.transform(scaled_model)

target = position_model['enc_player position'].values

['BLOCKED', 'MISSED', 'SCORED']
['BLOCKED', 'MISSED', 'SCORED']
['C', 'F', 'G', 'PF', 'PG', 'SF', 'SG']


/Users/nicholasbergeland/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/nicholasbergeland/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/nicholasbergeland/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [32]:
print(scaled_model.shape)
print(target.shape)

(6744, 6)
(6744,)


In [33]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(scaled_model, target,test_size=0.2,random_state=5)

In [34]:
from sklearn.ensemble import GradientBoostingClassifier

learning_rate = .75
gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)

gb_clf.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.75, loss='deviance', max_depth=2,
                           max_features=2, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=20,
                           n_iter_no_change=None, presort='auto',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [35]:
print("Learning rate: ", learning_rate)
print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))


Learning rate:  0.75
Accuracy score (training): 0.402
Accuracy score (validation): 0.355


In [36]:
print(position_model['player position'].value_counts())

print(position_model['enc_player position'].value_counts())

SG    2298
PF    1433
PG    1164
C     1004
SF     358
G      273
F      214
Name: player position, dtype: int64
6    2298
3    1433
4    1164
0    1004
5     358
2     273
1     214
Name: enc_player position, dtype: int64


In [37]:
shots[shots['player position'] == 'F']['shoot player'].unique()

array(['Paul Zipser'], dtype=object)

In [38]:
newChicagodata1=Chicago_shots.dropna()
newChicagodata1.isnull().sum()

NameError: name 'Chicago_shots' is not defined

In [15]:
newChicagodata = newChicagodata1.rename(columns={"player position":"POSITION","location x": "LOC_X", "location y": "LOC_Y", "shoot player":"PLAYER_NAME","current shot outcome":"EVENT_TYPE"})
newChicagodata

NameError: name 'newChicagodata1' is not defined

In [16]:
newChicagodata['EVENT_TYPE'].unique()

NameError: name 'newChicagodata' is not defined

In [17]:
print("Total made shots{}".format(newChicagodata[newChicagodata['EVENT_TYPE'] == 'SCORED'].shape))
print("Total missed shots{}".format(newChicagodata[newChicagodata['EVENT_TYPE'] == 'MISSED'].shape))

NameError: name 'newChicagodata' is not defined

In [18]:
newChicagodata['shot type'].unique()

NameError: name 'newChicagodata' is not defined

In [19]:
#visulization of what the current data looks like for shits and 
plt.figure(figsize=(10,10))

colors = np.where(newChicagodata['EVENT_TYPE']=='SCORED',"blue","red");
colors.shape
plt.subplot(121)
plt.title('made vs missed')
plt.xlabel('x_axis')
plt.ylabel('y_axis')
plt.scatter(newChicagodata.LOC_X,newChicagodata.LOC_Y,color=colors,alpha=0.5)

NameError: name 'newChicagodata' is not defined

<Figure size 720x720 with 0 Axes>

In [20]:
shots_taken_groups = newChicagodata.groupby('shot type')
for column, frame in shots_taken_groups:
    print('Column: {0}, Frame: {1}'.format(column, len(frame)))

NameError: name 'newChicagodata' is not defined

In [21]:
changedmissed = newChicagodata.replace(to_replace = "MISSED", value = "0")
changedmissed

NameError: name 'newChicagodata' is not defined

In [22]:
changedscored = changedmissed.replace(to_replace = "SCORED", value = "1")
changedblocked = changedscored.replace(to_replace = "BLOCKED",value = "2")
formattedChicago = changedblocked
formattedChicago

NameError: name 'changedmissed' is not defined

In [23]:
from sklearn.model_selection import train_test_split

y = np.array(formattedChicago['EVENT_TYPE']).astype(int)
X = formattedChicago.drop('EVENT_TYPE',1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)

NameError: name 'formattedChicago' is not defined

In [24]:
from sklearn import preprocessing 
locationoncourt = X_train[['LOC_X','LOC_Y']]

scaler = preprocessing.StandardScaler().fit(locationoncourt)
locationoncourt_scaled = scaler.transform(locationoncourt)

print(locationoncourt_scaled.mean(axis=0), locationoncourt_scaled.std(axis=0))
X_train.loc[:,0] = (locationoncourt_scaled[:,0]/locationoncourt_scaled[:,1])

/Users/nicholasbergeland/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices